In [9]:
import dspy
from dspy.datasets import Dataset
import pandas as pd
from sklearn.model_selection import train_test_split
import evaluate
from dspy.teleprompt import BootstrapFewShot, SignatureOptimizer
from dsp import LM
import requests
import json

In [10]:
class LoraLLama(LM):
    def __init__(self,model,**kwargs):
        self.model = model
        self.kwargs = kwargs
        self.provider = "default"
        self.history = []

        self.base_url = 'http://localhost:8000/dspy_optimize'
        self.config_url = 'http://localhost:8000/set_cfg'   

    def basic_request(self, prompt: str, **kwargs):
        headers = {
            "Content-Type":"application/json"
        }
        
        if kwargs:
            k_args = kwargs
        else:
            k_args = self.kwargs.copy()
        requests.post(self.config_url, headers=headers,json=k_args)
        
        data = {"text": prompt,
            "model": self.model
        }

        response = requests.post(self.base_url, headers=headers, json=data)
        response = response.json()['response']

        self.history.append({
            "prompt": prompt,
            "response": response,
            "kwargs": k_args,
        })
        return response

    def __call__(self, prompt, only_completed=True, return_sorted=False, **kwargs):
        response = self.request(prompt, **kwargs)

        completions = response

        return completions
model = LoraLLama(model='hej',temperature=1.56, max_tokens = 2000)
dspy.settings.configure(lm=model)

In [11]:
class SummarizeAsBulletedList(dspy.Module):
    def __init__(self):
        self.text_to_summary = dspy.ChainOfThought(text2summary)
        self.summary_to_bulleted_list = dspy.ChainOfThought(summary2bulleted_list)
    
    def forward(self, description: str) -> str:
        summary = self.text_to_summary(text=description).summary
        return self.summary_to_bulleted_list(summary=summary)

class text2summary(dspy.Signature):
    #"""Summarizes a given text."""
    """Summerar en given text."""

    text = dspy.InputField()
    summary = dspy.OutputField(prefix='Summering:',desc='Oftast mellan fem till tio meningar.')

class summary2bulleted_list(dspy.Signature):
    #"""Takes a summary and converts it to a bulleted list. 
    #The list has three topics: 'Medical History', 'Reasons for getting Care' & 'Measures taken by the caregiver'"""
    """Tar en summering och konverterar den till en punktlista.
    Listan har tre ämnen: '1. Sjukdomshistoria', '2. Sökorsaker' och '3. Åtgärder'."""
    
    summary = dspy.InputField()
    bulleted_list = dspy.OutputField()

In [12]:
data_train = []
data_dev = []
df = pd.read_parquet("../OpenAI/synthetic_229_corrected.parquet")
train, test = train_test_split(df,test_size = 0.05,random_state = 42)
for d, s in train.values:
    data_train.append(dspy.Example(description=d, summary=s).with_inputs("description"))
    
for d, s in test.values:
    data_dev.append(dspy.Example(description=d, summary=s).with_inputs("description"))


In [13]:
def validate_answer(example, pred, trace=None):
    rouge = evaluate.load("rouge")
    return rouge.compute(references=[example.summary], predictions=[pred.bulleted_list])["rougeL"]

In [14]:
teleprompter = SignatureOptimizer(metric=validate_answer, verbose = True)
kwargs = dict(num_threads=64, display_progress=True, display_table=0)
compiled = teleprompter.compile(SummarizeAsBulletedList(), devset=data_dev, eval_kwargs = kwargs)

[WARNING] SignatureOptimizer has been deprecated and replaced with COPRO.  SignatureOptimizer will be removed in a future release. 


  0%|          | 0/12 [77:21:24<?, ?it/s]


Iteration Depth: 1/3.
----------------
Predictor 1
i: D
p: 

Predictor 2
i: Tar en summering och konverterar den till en punktlista.
    Listan har tre ämnen: '1. Sjukdomshistoria', '2. Sökorsaker' och '3. Åtgärder'.
p: Bulleted List:

At Depth 1/3, Evaluating Prompt Candidate #1/13907 for Predictor 1 of 2.


Average Metric: 0.0 / 3  (0.0):  25%|██▌       | 3/12 [05:10<12:00, 80.06s/it] 

KeyboardInterrupt: 

In [7]:
compiled.save('../../data/dspy_test.json')

NameError: name 'compiled' is not defined